In [1]:
!pip install telebot

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [2]:
import random
import json
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification
from datasets import load_dataset
import telebot
from telebot import types
import torch
from torch.nn.functional import softmax

In [3]:
# Загружаем предобученную модель
saved_model_path = "./rubert_finetuned_DaNetQA/final_model"
model_name = "DeepPavlov/rubert-base-cased-conversational"
# Загрузить модель из сохранённого каталога:
loaded_model = BertForSequenceClassification.from_pretrained(saved_model_path, num_labels=2)
# Также загрузить токенизатор из сохранённого каталога:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [4]:
# Функция подготовки выборок для multiple choice task
def prepare_features(example):
    # Конструируем prompt для каждого варианта выбора
    inputs_data = example['question'] + '[SEP]' + example['passage']
    # Токенизируем с сохранением связи индексов
    inputs = tokenizer(inputs_data,
                       truncation=True,
                       max_length=512,
                       padding="max_length"
                       )
    label = example.get('label')
    if label is not None:
        inputs['labels'] = int(label)
    return inputs

In [5]:
# Обрабатываем датасеты
ds = load_dataset("DaNetQA")
ds_val_str = ds['validation']
length_val = len(ds_val_str)
ds_val_str

Dataset({
    features: ['question', 'passage', 'label', 'idx'],
    num_rows: 821
})

In [6]:
with open('bot_token.txt') as f:
    TOKEN = f.read().strip()

In [7]:
bot = telebot.TeleBot(TOKEN)
status = False
loaded_model.eval()
ds_val_str = ds_val_str.shuffle()

@bot.message_handler(commands=['start'])
def start(message):
    if not status:
        markup = types.InlineKeyboardMarkup(row_width=2)
        bt_true = types.InlineKeyboardButton(text="Да", callback_data="begin")
        bt_false = types.InlineKeyboardButton(text="Нет", callback_data="end")
        markup.add(bt_true, bt_false)
        bot.send_message(message.chat.id, f'Бот будет задавать вопросы, ваша задача выбрать правильный ответ. Да/Нет.' + \
                         '\nГотовы начать?', reply_markup=markup)

@bot.message_handler(func=lambda x: x in ["Да", "Нет", "Не отвечать на вопросы"])
def start_game(message):
    if status:
        ind = random.randrange(length_val)
        question = ds_val_str[ind]["question"]
        passage = ds_val_str[ind]["passage"]
        callback_data_yes = json.dumps({"action": "Да", "ind": ind})
        callback_data_no = json.dumps({"action": "Нет", "ind": ind})
        markup = types.InlineKeyboardMarkup(row_width=2)
        bt_true = types.InlineKeyboardButton(text="Да", callback_data=callback_data_yes)
        bt_false = types.InlineKeyboardButton(text="Нет", callback_data=callback_data_no)
        bt_cancel = types.InlineKeyboardButton(text="Закончить игру", callback_data="cancel")
        markup.add(bt_true, bt_false, bt_cancel)
        bot.send_message(message.chat.id, f'Прочитайте текс: \n"{passage}"\n\nОтветьте на вопрос: \n{question}',
                         reply_markup=markup)


@bot.callback_query_handler(func=lambda x: x.data)
def start_message(callback):
    global status
    if callback.data.find("action") > 0:
        data = json.loads(callback.data)
        ind = data.get("ind")
        answer_user = data.get("action")
        predictions = get_prediction(ind)
        answer_model = "Да" if predictions == 1 else "Нет"
        answer_true = "Да" if ds_val_str[ind] == "True" else "Нет"
        bot.send_message(
            callback.message.chat.id,
            text=f"Ваш ответ: {answer_user}.\n" +
            f"Нейросеть говорит {answer_model}.\n" +
            f"Правильный ответ: {answer_true}.\n"
        )
        start_game(callback.message)
    else:
        if callback.data == "cancel":
            status = False
        elif callback.data == "begin":
            status = True
            start_game(callback.message)
        elif callback.data == "end":
            status = False
    if not status:
        bot.send_message(callback.message.chat.id, text="До свидания.")



def get_prediction(ind):
    data = ds_val_str[ind]
    inputs_data = data['question'] + '[SEP]' + data['passage']
    inputs = tokenizer(inputs_data, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    predictions = outputs.logits.argmax().item()
    return predictions



bot.infinity_polling()

2025-06-01 23:02:59,672 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-06-01 23:02:59,673 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


![title](img/picture1.png)
![title](img/picture2.png)
